In [1]:
import pandas as pd
import requests
# API parameters
cycle = 2024  # Specify the election cycle
office = 'senate'  # Specify the office type
api_key = 'BVTfalQokoMzRLDe1oCsI1k9T4OxyttZ56GQFOhZ'  # Replace with your actual API key

# Base URL for the OpenFEC API
base_url = "https://api.open.fec.gov/v1/elections/"

states = [
    "AR", "NV", "MI", "OH", "PA", "WI", "MT", "NE", "TX"
]


senate_data = {}

for state in states:
    # Make the API call
    response = requests.get(f"{base_url}?cycle={cycle}&office={office}&state={state}&election_full=false&api_key={api_key}")

    # Check if the request was successful
    if response.status_code == 200:

        # Parse the JSON response
        data = response.json()

        
        df = pd.DataFrame(data['results'])
            # Store the DataFrame in the dictionary
        senate_data[state] = df
    else:           
        print(f"Failed to retrieve data for {state}: {response.status_code} - {response.text}")

print(senate_data)

{'AR':    candidate_election_year candidate_id  candidate_name candidate_pcc_id  \
0                     2024    S4AR00145  KASPARIAN, ANA        C00876268   

          candidate_pcc_name  cash_on_hand_end_period committee_ids  \
0  KASPER THE FRIENDLY GHOST                      0.0   [C00876268]   

  coverage_end_date incumbent_challenge_full         party_full  \
0              None               Challenger  PROGRESSIVE PARTY   

   total_disbursements  total_receipts  
0                  0.0             0.0  , 'NV':     candidate_election_year candidate_id              candidate_name  \
0                      2024    S8NV00156                ROSEN, JACKY   
1                      2024    S2NV00308                  BROWN, SAM   
2                      2024    S4NV00304             GUNTER, JEFF DR   
3                      2024    S4NV00270               MARCHANT, JIM   
4                      2024    S4NV00312                 GRADY, TONY   
5                      2024    S4NV00262 

In [2]:
# Create empty lists to store data
top_republicans = []

# Iterate through each seat
for state in senate_data:
    df = senate_data[state]
    
    # Get top Democrat
    dem_df = df[df['party_full'] == 'DEMOCRATIC PARTY']
    if not dem_df.empty:
        top_dem_receipts = dem_df['total_receipts'].max()
    else:
        top_dem_receipts = 0
        
    # Get top Republican
    rep_df = df[df['party_full'] == 'REPUBLICAN PARTY']
    if not rep_df.empty:
        top_rep = rep_df.loc[rep_df['total_receipts'].idxmax()]
        
        # Calculate distance from top Democrat
        distance = top_dem_receipts - top_rep['total_receipts']
        
        # Add to list with seat info
        top_rep_dict = top_rep.to_dict()
        top_rep_dict['state'] = state
        top_rep_dict['distance_from_max'] = distance
        top_republicans.append(top_rep_dict)

# Create final dataframe
top_rep_df = pd.DataFrame(top_republicans)
top_rep_df



,candidate_election_year,candidate_id,candidate_name,candidate_pcc_id,candidate_pcc_name,cash_on_hand_end_period,committee_ids,coverage_end_date,incumbent_challenge_full,party_full,total_disbursements,total_receipts,state,distance_from_max
0,2024,S2NV00308,"BROWN, SAM",C00845032,SAM BROWN FOR NEVADA,2249688.93,"[C00829481, C00845032]",2024-10-16,Challenger,REPUBLICAN PARTY,17751273.61,20000962.54,NV,18222311.79
1,2024,S4MI00595,"ROGERS, MICHAEL J",C00849810,ROGERS FOR SENATE,1888294.84,"[C00829457, C00849810]",2024-10-16,Open seat,REPUBLICAN PARTY,8764638.06,10652932.90,MI,35108862.15
2,2024,S4OH00192,"MORENO, BERNIE",C00837484,BERNIE MORENO FOR SENATE,2636429.38,"[C00775585, C00829499, C00837484]",2024-10-16,Challenger,REPUBLICAN PARTY,21447672.59,24237971.25,OH,60355067.01
3,2024,S2PA00661,"MCCORMICK, DAVE",C00851980,FRIENDS OF DAVE MCCORMICK,1675736.36,"[C00800623, C00829515, C00851980]",2024-10-16,Challenger,REPUBLICAN PARTY,26030736.92,27698652.75,PA,18441929.29
4,2024,S2WI00268,"HOVDE, ERIC D",C00870139,HOVDE FOR WISCONSIN,3610943.93,"[C00829523, C00870139]",2024-10-16,Challenger,REPUBLICAN PARTY,25920194.13,29531138.06,WI,13413368.89
5,2024,S4MT00183,"SHEEHY, TIM",C00844159,TIM SHEEHY FOR MONTANA,3877050.16,"[C00829465, C00844159]",2024-10-16,Challenger,REPUBLICAN PARTY,22284629.69,26161679.85,MT,57505118.13
6,2024,S2NE00094,"FISCHER, DEBRA S.",C00498907,DEB FISCHER FOR US SENATE,582814.19,[C00498907],2024-10-16,Incumbent,REPUBLICAN PARTY,6678159.73,5860776.48,NE,-5614447.48
7,2024,S2TX00312,"CRUZ, RAFAEL EDWARD TED",C00492785,TED CRUZ FOR SENATE,9565334.59,[C00492785],2024-10-16,Incumbent,REPUBLICAN PARTY,47067303.85,53252357.77,TX,26806934.40


In [3]:
top_rep_df = top_rep_df.sort_values('distance_from_max')
top_rep_df

,candidate_election_year,candidate_id,candidate_name,candidate_pcc_id,candidate_pcc_name,cash_on_hand_end_period,committee_ids,coverage_end_date,incumbent_challenge_full,party_full,total_disbursements,total_receipts,state,distance_from_max
6,2024,S2NE00094,"FISCHER, DEBRA S.",C00498907,DEB FISCHER FOR US SENATE,582814.19,[C00498907],2024-10-16,Incumbent,REPUBLICAN PARTY,6678159.73,5860776.48,NE,-5614447.48
4,2024,S2WI00268,"HOVDE, ERIC D",C00870139,HOVDE FOR WISCONSIN,3610943.93,"[C00829523, C00870139]",2024-10-16,Challenger,REPUBLICAN PARTY,25920194.13,29531138.06,WI,13413368.89
0,2024,S2NV00308,"BROWN, SAM",C00845032,SAM BROWN FOR NEVADA,2249688.93,"[C00829481, C00845032]",2024-10-16,Challenger,REPUBLICAN PARTY,17751273.61,20000962.54,NV,18222311.79
3,2024,S2PA00661,"MCCORMICK, DAVE",C00851980,FRIENDS OF DAVE MCCORMICK,1675736.36,"[C00800623, C00829515, C00851980]",2024-10-16,Challenger,REPUBLICAN PARTY,26030736.92,27698652.75,PA,18441929.29
7,2024,S2TX00312,"CRUZ, RAFAEL EDWARD TED",C00492785,TED CRUZ FOR SENATE,9565334.59,[C00492785],2024-10-16,Incumbent,REPUBLICAN PARTY,47067303.85,53252357.77,TX,26806934.40
1,2024,S4MI00595,"ROGERS, MICHAEL J",C00849810,ROGERS FOR SENATE,1888294.84,"[C00829457, C00849810]",2024-10-16,Open seat,REPUBLICAN PARTY,8764638.06,10652932.90,MI,35108862.15
5,2024,S4MT00183,"SHEEHY, TIM",C00844159,TIM SHEEHY FOR MONTANA,3877050.16,"[C00829465, C00844159]",2024-10-16,Challenger,REPUBLICAN PARTY,22284629.69,26161679.85,MT,57505118.13
2,2024,S4OH00192,"MORENO, BERNIE",C00837484,BERNIE MORENO FOR SENATE,2636429.38,"[C00775585, C00829499, C00837484]",2024-10-16,Challenger,REPUBLICAN PARTY,21447672.59,24237971.25,OH,60355067.01
